In [ ]:
!pip install konlpy
!pip install nltk
!pip install spacy
!pip install textblob

In [2]:
import re, os, nltk, konlpy, random, spacy, textblob
from konlpy.tag import Kkma, Okt

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
nltk.download('wordnet')
nltk.download("stopwords")
nltk.help.upenn_tagset()

In [6]:
from nltk.tokenize import sent_tokenize
print(sent_tokenize("Hi. Thank you for choosing Mundo. What seem to be problem today?"))

['Hi.', 'Thank you for choosing Mundo.', 'What seem to be problem today?']


In [7]:
#특징 찾기1
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
text="Me sorry, Cap'ain. You got the scurvy. Stage 12!"
x=word_tokenize(text)
print(x)
pos_tag(x)

['Me', 'sorry', ',', "Cap'ain", '.', 'You', 'got', 'the', 'scurvy', '.', 'Stage', '12', '!']


[('Me', 'NNP'),
 ('sorry', 'NN'),
 (',', ','),
 ("Cap'ain", 'NNP'),
 ('.', '.'),
 ('You', 'PRP'),
 ('got', 'VBD'),
 ('the', 'DT'),
 ('scurvy', 'NN'),
 ('.', '.'),
 ('Stage', 'NN'),
 ('12', 'CD'),
 ('!', '.')]

In [ ]:
!spacy download en

In [ ]:
#특징 찾기2 Lemmatizing
#표제어: 기본 사전형 단어(뿌리 단어를 찾아 개수 줄이기 위해)
#https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

text="Me sorry, Cap'ain. You got the scurvy. Stage 12!"
wnetx=word_tokenize(text)
wordnetlem=WordNetLemmatizer()

spacylem = spacy.load('en', disable=['parser', 'ner'])
print("wnet: ", [wordnetlem.lemmatize(z) for z in wnetx])
print("spcy: ", [z.lemma_ for z in spacylem(text)])
print("blob: ", [z.lemmatize() for z in TextBlob(text).words])

wnet:  ['Me', 'sorry', ',', "Cap'ain", '.', 'You', 'got', 'the', 'scurvy', '.', 'Stage', '12', '!']
spcy:  ['-PRON-', 'sorry', ',', "Cap'ain", '.', '-PRON-', 'get', 'the', 'scurvy', '.', 'stage', '12', '!']
blob:  ['Me', 'sorry', "Cap'ain", 'You', 'got', 'the', 'scurvy', 'Stage', '12']


In [ ]:
#특징 찾기3 Stemming
#어간: 어형 변화의 기초가 되는 부분

text="Me charge cancellation fee"
spanish_text="Yo cobrar cuota de cancelación"
x=word_tokenize(text)
spanish_x=word_tokenize(spanish_text)

#Porter: 자음+(모음+자음)의 반복+모음 연쇄, 국룰..?
#https://tartarus.org/martin/PorterStemmer/python.txt
print("Porter: ",[PorterStemmer().stem(w) for w in x])

#Lancaster: 더 '공격적', 처리속도빠르고 긴 단어에서 오류 많이나는 듯함
l=LancasterStemmer()
print("Lancaster: ", [LancasterStemmer().stem(w) for w in x])

#Snowball: 언어 지원
print("Snowball: ",[nltk.stem.SnowballStemmer('english').stem(w) for w in x])
print("Snowball_esp: ",[nltk.stem.SnowballStemmer('spanish').stem(w) for w in spanish_x]) #cobrar: 수집하다, yo cobro

#Regexp: 사용자 지정 정규식
from nltk.stem.regexp import RegexpStemmer as reg
print("Regexp: ",reg("e").stem(text).split())



Porter:  ['Me', 'charg', 'cancel', 'fee']
Lancaster:  ['me', 'charg', 'cancel', 'fee']
Snowball:  ['me', 'charg', 'cancel', 'fee']
Snowball_esp:  ['yo', 'cobr', 'cuot', 'de', 'cancel']
Regexp:  ['M', 'charg', 'cancllation', 'f']


In [10]:
from nltk.corpus import stopwords
#불용어: 의미분석에 기여하지 않는 단어(쓸데없는 단어)

text="I will bring them an opera of death."
print([w for w in word_tokenize(text) if w not in stopwords.words('english')])
print([w for w in word_tokenize(text) if w not in stopwords.words('english')+['death']])
type(stopwords.words('english'))

['I', 'bring', 'opera', 'death', '.']
['I', 'bring', 'opera', '.']


list

In [22]:
#Rose of Sharyn 가사
text='''Numb and broken, here I stand alone.
Wondering what were the last words I said to you.
It won't be long. We'll meet again.

What would I give to behold the smile, the face of love.
You never left me. The rising sun will always speak your name.

It won't be long, we'll meet again.
Your memory is never passing.
It won't be long, we'll meet again.
My love for you is everlasting.'''
a=sent_tokenize(text)
for x in a:
  print(x,'end')

Numb and broken, here I stand alone. end
Wondering what were the last words I said to you. end
It won't be long. end
We'll meet again. end
What would I give to behold the smile, the face of love. end
You never left me. end
The rising sun will always speak your name. end
It won't be long, we'll meet again. end
Your memory is never passing. end
It won't be long, we'll meet again. end
My love for you is everlasting. end


In [39]:
def integer_encoding(text):
  vocab = {} # 파이썬의 dictionary 자료형
  sentences = []
  stop_words = set(stopwords.words('english'))
  for i in text:
      sentence = word_tokenize(i) # 단어 토큰화를 수행합니다.
      result = []
      for word in sentence: 
          word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄입니다.
          if word not in stop_words: # 단어 토큰화 된 결과에 대해서 불용어를 제거합니다.
              if len(word) > 2: # 단어 길이가 2이하인 경우에 대하여 추가로 단어를 제거합니다.
                  result.append(word)
                  if word not in vocab:
                      vocab[word] = 0 
                  vocab[word] += 1
      sentences.append(result)
  return (sentences, vocab)
sentences=integer_encoding(a)[0]
vocab=sorted(integer_encoding(a)[1].items(),key=lambda x: x[1],reverse=True)
print(vocab)


[("n't", 3), ('long', 3), ("'ll", 3), ('meet', 3), ('love', 2), ('never', 2), ('numb', 1), ('broken', 1), ('stand', 1), ('alone', 1), ('wondering', 1), ('last', 1), ('words', 1), ('said', 1), ('would', 1), ('give', 1), ('behold', 1), ('smile', 1), ('face', 1), ('left', 1), ('rising', 1), ('sun', 1), ('always', 1), ('speak', 1), ('name', 1), ('memory', 1), ('passing', 1), ('everlasting', 1)]


In [47]:
def wordtoindex(vocab):
  word_to_index={}
  i=1
  for x in vocab:
    if x[1]>1:
      if x[0] not in word_to_index.keys():
        word_to_index[x[0]]=i
        i+=1
  return word_to_index
words_to_index=wordtoindex(vocab)
print(words_to_index)

{"n't": 1, 'long': 2, "'ll": 3, 'meet': 4, 'love': 5, 'never': 6}


In [29]:
from collections import Counter
words=[x for y in sentences for x in y]
print(Counter(words))
print(Counter(words)["alone"])
print(Counter(words).most_common(5))

Counter({"n't": 3, 'long': 3, "'ll": 3, 'meet': 3, 'love': 2, 'never': 2, 'numb': 1, 'broken': 1, 'stand': 1, 'alone': 1, 'wondering': 1, 'last': 1, 'words': 1, 'said': 1, 'would': 1, 'give': 1, 'behold': 1, 'smile': 1, 'face': 1, 'left': 1, 'rising': 1, 'sun': 1, 'always': 1, 'speak': 1, 'name': 1, 'memory': 1, 'passing': 1, 'everlasting': 1})
1
[("n't", 3), ('long', 3), ("'ll", 3), ('meet', 3), ('love', 2)]


In [49]:
wordtoindex(Counter(words).most_common(5))

{"'ll": 3, 'long': 2, 'love': 5, 'meet': 4, "n't": 1}

In [58]:
from nltk import FreqDist
import numpy as np
wordsfreq=FreqDist(words)
wordsfreq.most_common(5)
word_to_index={word[0]: index+1 for index, word in enumerate(wordsfreq.most_common(5))}
word_to_index

{"'ll": 3, 'long': 2, 'love': 5, 'meet': 4, "n't": 1}

In [128]:
vocab_size = 5
words_frequency = [w for w,c in word_to_index.items() if c >= vocab_size + 1] # 인덱스가 5 초과인 단어 제거
for w in words_frequency:
    del word_to_index[w] # 해당 단어에 대한 인덱스 정보를 삭제
word_to_index['OOV'] = len(word_to_index) + 1
print(word_to_index)

{"n't": 1, 'long': 2, "'ll": 3, 'meet': 4, 'love': 5, 'OOV': 6}


In [41]:
encoded = []
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            temp.append(word_to_index['OOV'])
    encoded.append(temp)
print(encoded)

[[6, 6, 6, 6], [6, 6, 6, 6], [1, 2], [3, 4], [6, 6, 6, 6, 6, 5], [6, 6], [6, 6, 6, 6, 6], [1, 2, 3, 4], [6, 6, 6], [1, 2, 3, 4], [5, 6]]


In [110]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=6,oov_token='OOV') #상위 5개, OOV는 자동으로 1
tokenizer.fit_on_texts(sentences)
tfsent=tokenizer.texts_to_sequences(sentences)
print([x for x in tokenizer.word_index.items() if x[1]<6])
print(tokenizer.word_counts)
print(tfsent)

[('OOV', 1), ("n't", 2), ('long', 3), ("'ll", 4), ('meet', 5)]
OrderedDict([('numb', 1), ('broken', 1), ('stand', 1), ('alone', 1), ('wondering', 1), ('last', 1), ('words', 1), ('said', 1), ("n't", 3), ('long', 3), ("'ll", 3), ('meet', 3), ('would', 1), ('give', 1), ('behold', 1), ('smile', 1), ('face', 1), ('love', 2), ('never', 2), ('left', 1), ('rising', 1), ('sun', 1), ('always', 1), ('speak', 1), ('name', 1), ('memory', 1), ('passing', 1), ('everlasting', 1)])
[[1, 1, 1, 1], [1, 1, 1, 1], [2, 3], [4, 5], [1, 1, 1, 1, 1, 1], [1, 1], [1, 1, 1, 1, 1], [2, 3, 4, 5], [1, 1, 1], [2, 3, 4, 5], [1, 1]]


In [72]:
def padding(sent):
  maxlength=max(len(item) for item in sent)
  for item in sent:
    while len(item)<maxlength:
      item.append(0)
  return sent
np.array(padding(tfsent))

array([[1, 1, 1, 1, 0, 0],
       [1, 1, 1, 1, 0, 0],
       [2, 3, 0, 0, 0, 0],
       [4, 5, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1],
       [1, 1, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 0],
       [2, 3, 4, 5, 0, 0],
       [1, 1, 1, 0, 0, 0],
       [2, 3, 4, 5, 0, 0],
       [1, 1, 0, 0, 0, 0]])

In [112]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
tfsent=tokenizer.texts_to_sequences(sentences)
pad_sequences(tfsent,padding='post',maxlen=5,truncating='post',value=111)

array([[  1,   1,   1,   1, 111],
       [  1,   1,   1,   1, 111],
       [  2,   3, 111, 111, 111],
       [  4,   5, 111, 111, 111],
       [  1,   1,   1,   1,   1],
       [  1,   1, 111, 111, 111],
       [  1,   1,   1,   1,   1],
       [  2,   3,   4,   5, 111],
       [  1,   1,   1, 111, 111],
       [  2,   3,   4,   5, 111],
       [  1,   1, 111, 111, 111]], dtype=int32)

In [116]:
word2index={}
i=0
for x in nltk.word_tokenize(a[0]):
  if x not in word2index.keys():
    word2index[x]=i
    i+=1
print(word2index)

{'Numb': 0, 'and': 1, 'broken': 2, ',': 3, 'here': 4, 'I': 5, 'stand': 6, 'alone': 7, '.': 8}


In [118]:
def one_hot_encoding(word, word2index):
       one_hot_vector = [0]*(len(word2index))
       index=word2index[word]
       one_hot_vector[index]=1
       return one_hot_vector

In [119]:
one_hot_encoding("and",word2index)

[0, 1, 0, 0, 0, 0, 0, 0, 0]

In [124]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
t=Tokenizer()
t.fit_on_texts([a[0]])
t.word_index

{'alone': 7, 'and': 2, 'broken': 3, 'here': 4, 'i': 5, 'numb': 1, 'stand': 6}

In [127]:
subtext='here stand alone'
to_categorical(t.texts_to_sequences([subtext])[0])
#일반적으론 단어간 유사도 부여 불가 고양이<->호랑이 vs 고양이<->고정

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
#https://babynames.com/top-baby-names-of-2020

text='''1UP OliverUP Amelia/Emilia
2DOWN LiamDOWN Charlotte
3SAME TheodoreUP Aurora
4UP EthanDOWN Violet
5BIGUP Aidan/AidenUP Olivia/Alivia
6UP BenjaminSAME Ava
7DOWN DeclanDOWN Aria/Arya
8BIGUP GabrielUP Luna
9DOWN Finn/FynnUP Isla
10UP ElijahDOWN Hazel
11DOWN HenrySAME Scarlett
12DOWN OwenUP Maeve
13DOWN CalebDOWN Nora/Norah
14DOWN AlexanderBIGUP Sophia/Sofia
15DOWN Grayson/GreysonBIGUP Chloe
16UP SebastianUP Freya/Freja
17DOWN LeoUP Abigail
18DOWN LeviBIGUP Audrey
19UP JasperBIGUP Grace
20UP Lucas/LukasDOWN Adelaide
21SAME Jackson/JaxonSAME Lily/Lilly
22BIGDOWN EmmettBIGUP Isabella
23BIGUP Miles/MylesBIGDOWN Vivian
24DOWN EverettDOWN Eleanor
25UP FelixBIGUP Ella
26UP LandonBIGUP Ophelia
27BIGUP AtticusUP Adeline
28BIGUP MiloBIGDOWN Genevieve
29UP JackDOWN Ivy
30BIGUP TheoBIGDOWN Lucy
31BIGDOWN Elliot/ElliottDOWN Emma
32DOWN ArcherBIGUP Hannah/Hanna
33DOWN SilasBIGUP Iris
34BIGUP IsaacBIGUP Esme
35UP ArthurBIGDOWN Penelope
36BIGUP EliasDOWN Clara
37BIGDOWN WilliamUP Cora
38DOWN WyattDOWN Eloise
39NEW CaydenSAME Mila
40DOWN NoahUP Astrid
41BIGUP MatteoBIGUP Arabella
42BIGUP SamuelBIGDOWN Naomi
43DOWN HudsonDOWN Elizabeth
44UP HarrisonNEW Isabelle
45UP AugustBIGUP Evangeline
46NEW Alastair/AlistairBIGDOWN Ruby
47BIGDOWN GavinUP Sadie
48BIGUP JacobUP Mia
49DOWN LincolnBIGUP Maya/Maia
50BIGUP AaronNEW Keira
51BIGUP EzraUP Wren
52DOWN AtlasBIGUP Autumn
53UP ColeBIGUP Gemma/Jemma
54BIGUP Xander/ZanderBIGUP Ada
55BIGUP TobiasNEW Delilah
56BIGDOWN HoldenBIGDOWN Rosalie
57DOWN NathanielBIGDOWN Lorelei
58BIGUP DominicDOWN Eliza
59BIGUP IsaiahBIGDOWN Anastasia
60BIGUP Adrian/AdrienDOWN Josephine
61BIGDOWN RhysBIGDOWN Claire/Clare
62UP GideonDOWN Rose
63UP DanielBIGDOWN Lydia
64UP LukeBIGUP Kaia/Kaya
65BIGDOWN JamesBIGUP Saoirse
66NEW Connor/ConorUP Layla
67BIGUP XavierBIGDOWN Lila/Lilah
68BIGDOWN CallumDOWN Margot/Margo
69UP ArloBIGUP Phoebe
70BIGDOWN LoganBIGDOWN Stella
71DOWN AxelBIGUP Zoe
72NEW EdwardBIGDOWN Juliet/Juliette
73BIGUP ThomasBIGUP Aurelia
74NEW EricBIGUP Daisy
75NEW MatthewUP Athena
76SAME ZacharyDOWN Thea
77BIGDOWN AsherBIGDOWN Juniper
78BIGUP EliBIGDOWN Alice
79UP BeauDOWN Madeline
80BIGDOWN RomanBIGDOWN Natalie
81BIGDOWN NolanBIGDOWN Fiona
82DOWN NicholasUP Sienna
83UP LachlanNEW Eliana
84BIGDOWN NathanNEW Willow
85BIGDOWN RonanNEW Vera
86UP AdamBIGDOWN Matilda
87BIGDOWN JonahDOWN Emily
88NEW AceUP Olive
89BIGDOWN AndrewUP Dahlia
90UP ColtonNEW Anna
91NEW AbelNEW Mabel
92BIGDOWN CharlesNEW Eva
93NEW MichaelBIGDOWN Evelyn
94NEW MalachiNEW Leilani
95NEW VincentBIGDOWN Harper
96NEW CaspianBIGDOWN Daphne
97NEW OscarNEW Sophie
98UP IanNEW Amara
99BIGDOWN RowanDOWN Brielle
100NEW Luca/LukaDOWN Aaliyah'''
import re
j=re.findall("[A-Z][a-z]+/?",text)
a=1
t=[]
tlen=[]
for x in j:
  if '/' not in x:    #불필요한 단어 제거 (원래는 등장빈도가 낮거나 길이가 짧은 경우/불용어일 경우 제거할 때가 많음)
    if a==0:
       a=1
    elif a==1:
       a=0
    t.append([x,a])
    if len(x)>=6: #불필요한 단어 제거 (길이가 짧을 때)
       tlen.append([x.lower(),a])
print(t[:5])
print(tlen[:5])

[['Oliver', 0], ['Emilia', 1], ['Liam', 0], ['Charlotte', 1], ['Theodore', 0]]
[['oliver', 0], ['emilia', 1], ['charlotte', 1], ['theodore', 0], ['aurora', 1]]


In [ ]:
X,y=zip(*t)
print(X[:5])
print(y[:5])

('oliver', 'emilia', 'liam', 'charlotte', 'theodore')
(0, 1, 0, 1, 0)


In [ ]:
import pandas as pd
df=pd.DataFrame(t,columns=['name','01'])
df.head()

,name,01
0,oliver,0
1,emilia,1
2,liam,0
3,charlotte,1
4,theodore,0


In [ ]:
X=df.iloc[:,0]
y=df.iloc[:,1]
print(tuple(X)[:5])
print(tuple(y)[:5])

('oliver', 'emilia', 'liam', 'charlotte', 'theodore')
(0, 1, 0, 1, 0)


In [ ]:
n_of_train = int(len(X) * 0.8)
X_test = X[n_of_train:] 
y_test = y[n_of_train:] 
X_train = X[:n_of_train] 
y_train = y[:n_of_train]
print(len(X_test))
print(len(X_train))

40
160


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=1234)
print(len(X_test))
print(len(X_train))

40
160


In [129]:
import konlpy
from konlpy.tag import Kkma, Okt
kkma=Kkma()
okt=Okt()

In [ ]:
print(okt.morphs("혈액 샘플 필요하다!"))
print(okt.pos("혈액 샘플 필요하다!"))

['혈액', '샘플', '필요하다', '!']
[('혈액', 'Noun'), ('샘플', 'Noun'), ('필요하다', 'Adjective'), ('!', 'Punctuation')]


In [ ]:
print(okt.morphs("제세동기르르릃르르릃르르릃."))
print(okt.pos("제세동기르르릃르르릃르르릃."))

['제세동기', '르르', '릃르르릃르르릃', '.']
[('제세동기', 'Noun'), ('르르', 'Noun'), ('릃르르릃르르릃', 'Noun'), ('.', 'Punctuation')]


In [ ]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
!pip install soynlp

In [ ]:
!pip install customized_konlpy

In [ ]:
text="혈액 샘플 필요하다!".replace(" ",'')
texttwo="외않되?"
from pykospacing import Spacing
from hanspell import spell_checker
spacing=Spacing()
print(text)
print(spacing(text))
print(spell_checker.check(texttwo).checked)

혈액샘플필요하다!
혈액 샘플 필요하다!
왜 안돼?


In [167]:
#Soynlp 응집확률(문자를 문자열 단위로 분리 후 순서대로 추가하면서 가장 높은 확률이 단어로 나올 가능성이 높다!)
#비지도 학습 기반(데이터 필요)
#아래와 같이 바뀌어가는 언어문화에 적응하기 위한 접근
#카카오톡은 11년부터 자주 쓰이기 시작한 단어
print(Okt().morphs("무야호... 야호... 그만큼 신나시는 거지~")) 
print(Okt().morphs("카카오톡으로 만나는 단골고객 카카오톡 스토어, 지금 새로운 단골고객을 만들어 보세요.")) 

['무', '야호', '...', '야호', '...', '그만큼', '신나시는', '거지', '~']
['카카오', '톡', '으로', '만나는', '단골', '고객', '카카오', '톡', '스토어', ',', '지금', '새로운', '단골', '고객', '을', '만들어', '보세요', '.']


In [134]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")
corpus = DoublespaceLineCorpus("2016-10-20.txt")
len(corpus)
word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 1.906 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [165]:
print(word_score_table["카카"].cohesion_forward)
print(word_score_table["카카"].right_branching_entropy)

0.1386575208231259
-0.0


In [166]:
print(word_score_table["카카오"].cohesion_forward)
print(word_score_table["카카오"].right_branching_entropy)

0.3723674540331444
2.1028544930905926


In [164]:
print(word_score_table["카카오톡"].cohesion_forward)
print(word_score_table["카카오톡"].right_branching_entropy)

0.40341675456207704
2.2478442027083916


In [171]:
from soynlp.tokenizer import LTokenizer
from soynlp.tokenizer import MaxScoreTokenizer
#LTokenizer는 어절 분리(L,R)
#MaxScore는 띄어쓰기가 안되어있을 때
scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
l_tokenizer = LTokenizer(scores=scores)
print(l_tokenizer.tokenize("카카오톡으로 만나는 단골고객 카카오톡 스토어, 지금 새로운 단골고객을 만들어 보세요.", flatten=False))
maxscore_tokenizer = MaxScoreTokenizer(score  s=scores)
print(maxscore_tokenizer.tokenize("카카오톡으로만나는단골고객카카오톡스토어,지금새로운단골고객을만들어보세요."))

[('카카오톡', '으로'), ('만나는', ''), ('단골고객', ''), ('카카오톡', ''), ('스토어', ','), ('지금', ''), ('새로운', ''), ('단골고객을', ''), ('만들어', ''), ('보세요.', '')]
['카카오톡', '으로', '만나는', '단골', '고객', '카카오톡', '스토어', ',', '지금', '새로운', '단골', '고객', '을만', '들어', '보세요.']


In [192]:
from soynlp.normalizer import *
text1="ㅋㅋㅋㅋㅋ카톡보셈"
text2="ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ카톡보셈"
text3="크크크크크이따톡보셈"
example=[text1,text2,text3]
for x in example:
  print(emoticon_normalize(x,num_repeats=3))
  print(repeat_normalize(x,num_repeats=2))
  print()

ㅋㅋㅋㅏ톡보셈
ㅋㅋ카톡보셈

ㅋㅋㅋㅏ톡보셈
ㅋㅋ카톡보셈

크크크이따톡보셈
크크이따톡보셈



In [191]:
from ckonlpy.tag import Twitter #여기엔 Okt가 없는 듯..합니다
twitter=Twitter()
print(twitter.morphs('무야호... 야호... 그만큼 신나시는 거지~'))
twitter.add_dictionary('무야호','Noun')
print(twitter.morphs('무야호... 야호... 그만큼 신나시는 거지~'))

/usr/local/lib/python3.7/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


['무', '야호', '...', '야호', '...', '그', '만큼', '신', '나시', '는', '거지', '~']
['무야호', '...', '야호', '...', '그', '만큼', '신', '나시', '는', '거지', '~']


In [ ]:
#PDF 다운로드 링크: https://drive.google.com/file/d/1ozsOG9U8jNWePCVf9cbV5p7_hNtcF6eH/view?usp=sharing